## Control the requirements

In [1]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.3.1'

In [5]:
import sklearn

## Here, first of all I tried it for given dataset.

In [16]:
from textgenrnn import textgenrnn

textgen = textgenrnn()
textgen.generate()

Can anyone else have a stranger become home and games answer



In [10]:
textgen.train_from_file('datasets/hacker_news_2000.txt', num_epochs=1)
textgen.generate()

2,000 texts collected.
Training on 83,501 character sequences.
652/652 [==============================] - ETA: 0s - loss: 1.7692####################
Temperature: 0.2
####################
Google company is a startup of a startup

A company is not starting on the startup of the startup

A startup of the Universe of Startup

####################
Temperature: 0.5
####################
I think I want to an and show sential end of servers

What a simple source controvern swat

Good for a data and locker to let your state transportation old

####################
Temperature: 1.0
####################
Announces explicate car from 311 felting 9115 loop is not just end on Sony to Gotbap Use/I Ignogie

Hellows $122

Had YC (2010)

652/652 [==============================] - 173s 265ms/step - loss: 1.7692
How a standary of the survey startup developer



**Now, I found SMS Spam Collection in English data set from https://blog.cambridgespark.com/50-free-machine-learning-datasets-natural-language-processing-d88fb9c5c8da**

In [18]:
model = textgenrnn()

In [5]:
model.train_from_file('smsspamcollection/SMSSpamCollection.txt', num_epochs=2)
model.generate()

5,574 texts collected.
Training on 477,088 character sequences.
Epoch 1/2
3727/3727 [==============================] - ETA: 0s - loss: 1.7604####################
Temperature: 0.2
####################
hamNo shopping and and send the way to get the stread of the man will be sorry to get the be and then you can get the shit of mate. I can get a little now will be a good is a good morning. I can get the stread and i want to stop a £2000 prize GUARANT to 8770666666 110p/more call 087110002000000 per

hamGood message. I can get a good call of a good and start in a minutes of the mobile and i was then you have a good day. Thats you are trying to come to be a good teach and i send you to get the message. I can get the way to anyone and the little story to 8700 and send a £100 prize GUARANT to 8700

hamHave you gonna get the man to me and start and the boy is a good teach and the new year day. Call 09066696666 and I will be one to your and i have a go to change the mates and watch that was the 

**Also, I found Nietzsche text data set from https://www.kaggle.com/pankrzysiu/nietzsche-texts but it is very long so I used first part of it.**

In [9]:
model.train_from_file('nietzsche copy.txt', num_epochs=1)
model.generate()

5,930 texts collected.
Training on 377,315 character sequences.
2947/2947 [==============================] - ETA: 0s - loss: 1.5512####################
Temperature: 0.2
####################
the only and present is the most problem of the hand and the problem of the strength and the strange and strange and strange and and as the most strength and the for the strange of the strength of the present of the strength of the strength of the more present of the more will an art of the because

and and desire of the strange of the present of the most strange and still; which is always and and the problem of the most part of the present and end and the strange of the more strange of the more and strange and the strange of the problem and strange of the problem of the morality of the more a

and strange and still; and the present of the present of the more strange and from the strength and the strength and strange of the present of the hand of the problem of the problem and strength of the more a

**Here, I used a data set by using scraping in class.**

In [1]:
import requests
import zipfile
from tqdm import tqdm
from collections import Counter
import numpy as np

In [2]:
def download_data():
    url = "https://americanenglish.state.gov/files/ae/resource_files/the_tell-tale_heart_0.pdf"
    r = requests.get(url, stream=True)
    
    bar = tqdm (
        total = int(r.headers["Content-Length"]),
        initial = 0,
        unit = 'B',
        unit_scale = True,
    )
    
    with open("data.zip", "wb") as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                bar.update(1024)
    bar.close()

In [12]:
def fetch_words():
    words = []
    with open("text.txt", "r") as f:
        for line in f:
            for word in line.strip().split(" "):
                words.append(word)
                
    return words

In [13]:
def build_data(words, n_words):
    freq = [['UNKNOWN', -1]]
    
    c = Counter(words)
    pairs = list(c.items())
    pairs_filtered = sorted(pairs, key=lambda x: -x[1])[:n_words-1] 
    for word, count in pairs_filtered: 
        freq.append([word, count])
        
    dictionary = dict()
    for word, _ in freq:
        dictionary[word] = len(dictionary)
        
    data = list()
    unknown_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNKNOWN']
            unknown_count += 1
        data.append(index)
        
    freq[0][1] = unknown_count
    
    inverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, freq, dictionary, inverse_dictionary

In [14]:
download_data()
words = fetch_words()

vector_dim = 300
epochs = 200000
vocabulary_upper_limit = 10000
data, freq, dictionary, inverse_dictionary = build_data(words, vocabulary_upper_limit)

162kB [00:00, 621kB/s]                            


In [19]:
model.train_from_file('text.txt', num_epochs=1)
model.generate()

69 texts collected.
Training on 10,678 character sequences.
83/83 [==============================] - ETA: 0s - loss: 1.6218####################
Temperature: 0.2
####################






####################
Temperature: 0.5
####################
I had passed a small character of my dead at the person of the night of the shades, there is no interpelant of the front upon and what the would a betweeve the opening of heards, nothing with the officient of the mistake and heard. I was nothered and had to lead the son. I dont thanking out of my h





####################
Temperature: 1.0
####################


While Build! A still, his doorpry. I need to shares, increased, dusted the before a statie I laughed.



83/83 [==============================] - 30s 359ms/step - loss: 1.6218




**Model**

In [25]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Reshape, dot
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence

import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [26]:
sampling_table = sequence.make_sampling_table(vocabulary_upper_limit)
couples, labels = skipgrams(data, vocabulary_upper_limit, window_size=3, sampling_table=sampling_table)
word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

In [34]:
len(couples)

1932

In [27]:
input_target, input_context = Input(shape=(1)), Input(shape=(1))

embedding = Embedding(vocabulary_upper_limit, vector_dim, input_length=1, name='embedding')

target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)

context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

dot_product = dot([target, context],  axes=1)
dot_product = Reshape((1,))(dot_product)

output = Dense(1, activation='sigmoid')(dot_product)

model = Model(inputs=[input_target, input_context], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam')

In [28]:
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 300)       3000000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 300, 1)       0           embedding[0][0]       

In [39]:
batch_size = 40960
epochs = 1
for itr in range(epochs):
    for batch_num in range(len(word_target)//batch_size):
        myslice = slice(batch_num*batch_size,(batch_num+1)*batch_size)
          
        batch_inputs = np.array(word_target[myslice]).reshape(batch_size,1)
        batch_contexts = np.array(word_context[myslice]).reshape(batch_size,1)
        batch_labels = np.array(labels[myslice]).reshape(batch_size,1)

        loss = model.train_on_batch([batch_inputs,batch_contexts],batch_labels)
        print(f"Epoch: {itr}\tBatch Num:{batch_num}/{len(word_target)//batch_size}\tloss={loss}")